In [1]:
import tensorflow as tf
import skimage.io
import skimage.color
import numpy as np
from pprint import pprint

In [2]:
import read_cifar10 as cf10

#@read_data.restartable
def cifar10_dataset_generator(dataset_name, batch_size, restrict_size=1000):
    assert dataset_name in ['train', 'test']
    assert batch_size > 0 or batch_size == -1  # -1 for entire dataset
    
    X_all_unrestricted, y_all = (cf10.load_training_data() if dataset_name == 'train'
                                 else cf10.load_test_data())
    
    actual_restrict_size = restrict_size if dataset_name == 'train' else int(1e10)
    X_all = X_all_unrestricted[:actual_restrict_size]
    data_len = X_all.shape[0]
    batch_size = batch_size if batch_size > 0 else data_len
    
    X_all_padded = np.concatenate([X_all, X_all[:batch_size]], axis=0)
    y_all_padded = np.concatenate([y_all, y_all[:batch_size]], axis=0)
    
    for slice_i in range(math.ceil(data_len / batch_size)):
        idx = slice_i * batch_size
        #X_batch = X_all_padded[idx:idx + batch_size]
        X_batch = X_all_padded[idx:idx + batch_size]*255  # bugfix: thanks Zezhou Sun!
        y_batch = np.ravel(y_all_padded[idx:idx + batch_size])
        yield X_batch.astype(np.uint8), y_batch.astype(np.uint8)

cifar10_dataset_generators = {
    'train': cifar10_dataset_generator('train', 1000),
    'test': cifar10_dataset_generator('test', -1)
}


In [3]:
#Load cifar-10 data
cf10_tr=cf10.load_training_data()
cf10_tr_img=cf10_tr[0]
cf10_tr_label = cf10_tr[1]

cf10_test=cf10.load_test_data()
cf10_test_img=cf10_test[0]
cf10_test_label = cf10_test[1]

In [4]:
cf10_test_img_gray=(cf10_test_img[:,:,:,0]+cf10_test_img[:,:,:,1]+cf10_test_img[:,:,:,2])/3.
cf10_tr_img_gray=(cf10_tr_img[:,:,:,0]+cf10_tr_img[:,:,:,1]+cf10_tr_img[:,:,:,2])/3.
cf10_tr_vec=np.zeros((50000,1024))
cf10_test_vec=np.zeros((10000,1024))

In [5]:
import skimage.io
def img2block(im):
    '''
    Image patching code. It patches a given RGB image into 32x32 blocks and returns a 4D array with size 
    [number_of_patches,32,32,3]
    '''
    im = im.astype(np.float32)
    row,col,color = im.shape
    im_bl=np.zeros((int(row*col/1024),32,32,3)).astype(np.float32)
    count=0
    for i in range(0,row-row%32,32):
        for j in range(0,col-col%32,32):
            im_bl[count,:,:,:]=im[i:i+32,j:j+32,:]
            count = count +1
    im_bl=im_bl/255.
    return im_bl

def block2img(img_blocks,img_size):
    '''
    Function for reconstructing the image back from patches
    '''
    row,col = img_size
    img=np.zeros((row,col,3)).astype(np.float32)
    n,k,l,c=img_blocks.shape
                 
    for i in range(0,int(row/k)):
        for j in range(0,int(col/k)):
            img[i*k:(i+1)*k,j*l:(j+1)*l,:]=img_blocks[int(i*col/k+j),:,:,:]
    return img

#Get the patches of lena image
lena_img = skimage.io.imread('../test_img/lena512color.tiff')
lena_32=img2block(lena_img)
        
        

In [6]:
def convert2uint8(img):
    img[img>255]=255
    img[img<0]=0
    return img.astype(np.uint8)

# Part-1 - MLP

In [8]:
x_tr = cf10_tr_vec.astype(np.float32)/255.
x_test = cf10_test_vec.astype(np.float32)/255.
x_test=x_test[:1000,:]

In [9]:

def mlp1(x, hidden_sizes, activation_fn=tf.nn.relu,dropout_rate=1.0,std_dev=1.0):
    if not isinstance(hidden_sizes, (list, tuple)):
        raise ValueError("hidden_sizes must be a list or a tuple")
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    for k in range(len(hidden_sizes)-1):
        layer_name="weights"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[k]])
            b = tf.get_variable('b', shape=[hidden_sizes[k]])
            x = activation_fn(tf.matmul(x, W) + b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes[-1]])
        return tf.matmul(x, W) + b

In [10]:
import tensorflow as tf

def mlp2(x, hidden_sizes_1,hidden_sizes_2, activation_fn=tf.nn.relu,dropout_rate=1.0,std_dev=1.0,cons_mult=1):
    scope_args = {'initializer': tf.random_normal_initializer(stddev=std_dev)}
    for k in range(len(hidden_sizes_1)-1):
        layer_name="weights_enc"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_1[k]])
            b = tf.get_variable('b', shape=[hidden_sizes_1[k]])
            x = activation_fn(tf.matmul(x, W) + cons_mult*b)
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer_enc', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_1[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes_1[-1]])
        x = activation_fn(tf.matmul(x, W) + cons_mult*b)
        
    x_quant = tf.round(x*255.)/255.
    
    for k in range(len(hidden_sizes_2)-1):
        layer_name="weights_dec"+str(k)
        #FC layers
        with tf.variable_scope(layer_name, **scope_args):
            W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_2[k]])
            b = tf.get_variable('b', shape=[hidden_sizes_2[k]])
            x = activation_fn(tf.matmul(x, W) + cons_mult*b)
            x_quant=(activation_fn(tf.matmul(x_quant, W) + cons_mult*b))
            #Dropout before the last layer
            x = tf.nn.dropout(x, keep_prob=dropout_rate)
    #Softmax layer
    with tf.variable_scope('outlayer_dec', **scope_args):
        W = tf.get_variable('W', shape=[x.shape[-1], hidden_sizes_2[-1]])
        b = tf.get_variable('b', shape=[hidden_sizes_2[-1]])
    
    return (tf.matmul(x, W) + cons_mult*b,tf.matmul(x_quant, W) + cons_mult*b)

In [13]:
import pprint

def test_classification(model_function, learning_rate=0.1):

    with tf.Graph().as_default() as g:
        # where are you going to allocate memory and perform computations
        with tf.device("/gpu:0"):
            
            # define model "input placeholders", i.e. variables that are
            # going to be substituted with input data on train/test time
            x_ = tf.placeholder(tf.float32, [None, 1024])
            #y_logits = model_function(x_)
            y_logits,x_recon = model_function(x_)

            
            # naive implementation of loss:
            # > losses = y_ * tf.log(tf.nn.softmax(y_logits))
            # > tf.reduce_mean(-tf.reduce_sum(losses, 1))
            # can be numerically unstable.
            #
            # so here we use tf.nn.softmax_cross_entropy_with_logits on the raw
            # outputs of 'y', and then average across the batch.
            
            loss = tf.reduce_mean(tf.subtract(x_,y_logits)**2)
            loss2=tf.reduce_mean(tf.subtract(x_,x_recon)**2)
            train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
            
            #y_pred = tf.argmax(tf.nn.softmax(y_logits), dimension=1)
            #correct_prediction = tf.equal(y_pred, tf.argmax(y_, 1))
            #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with g.as_default(), tf.Session() as sess:
        # that is how we "execute" statements 
        # (return None, e.g. init() or train_op())
        # or compute parts of graph defined above (loss, output, etc.)
        # given certain input (x_, y_)
        #sess.run(tf.initialize_all_variables())
        sess.run(tf.global_variables_initializer())
        
        # train
        ids=[i for i in range(100)]
        for iter_i in range(100001):
            batch_xs = x_tr[ids,:] 
            ids=[(ids[0]+100+i)%x_tr.shape[0] for i in range(100)]
            sess.run(train_step, feed_dict={x_: batch_xs})
            
            # test trained model
            if iter_i % 1000 == 0:
                tf_feed_dict = {x_: batch_xs}
                loss_val = sess.run(loss, feed_dict=tf_feed_dict)
                print('iteration %d\t train mse: %.3f\t'%(iter_i,loss_val))
                if iter_i%5000 == 0:
                    
                    loss_val_test = sess.run(loss, feed_dict={x_:x_test})
                    loss_val2_test = sess.run(loss2, feed_dict={x_:x_test})
                    print('iteration %d\t TEST MSE: %.3f\t %.3f\t'%(iter_i,loss_val_test,loss_val2_test))
                
                    
                    x_from_tr=sess.run(y_logits, feed_dict={x_:batch_xs[:5,:].reshape([-1,1024])})
                    x_from_test=sess.run(y_logits, feed_dict={x_:x_test[0:5,:].reshape([-1,1024])})
                    
                    
                
#test_classification(lambda x: mlp1(x, [850,700,500,700,850,1024],
#                                  activation_fn=tf.nn.relu,std_dev=1e-1), learning_rate=1e-3)

test_classification(lambda x: mlp2(x, [850,700,500],[700,850,1024],
                                  activation_fn=tf.nn.relu,std_dev=1,cons_mult=0.5), learning_rate=1e-3)

NameError: name 'train_step' is not defined